In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import random

In [3]:
civil_df = pd.read_csv("./data/civil_comments.csv")
civil_df_thomas = pd.read_csv("./data/all_data_with_identities.csv")
train_identities_df = pd.read_csv("./data/train_identities.txt", header=None)
train_identities_lists = train_identities_df.values.tolist()
test_identities_df = pd.read_csv("./data/test_identities.txt", header=None)
test_identities = test_identities_df.values.tolist()

In [4]:
train_identities = []
for ident in train_identities_lists:
    train_identities.append(ident[0])
#train_identities

In [5]:
print(sum(civil_df_thomas['toxicity'] < 0.5))
print(civil_df_thomas.shape)

397206
(448000, 59)


In [6]:
df_non_toxic = civil_df_thomas[civil_df_thomas['toxicity']<0.5]

In [7]:
df_non_toxic.shape

(397206, 59)

In [8]:
df_non_toxic['comment_text'].head()

9     Trump could accidently push a button and Canad...
10    He's considered a good candidate for a cyber-s...
11    Lela, you admit no records exist to support yo...
16         Shouldn't your handle be Republic of Uranus?
19    The guy commits espionage and the lefties at H...
Name: comment_text, dtype: object

In [9]:
len(civil_df_thomas)

448000

In [10]:
sentences = []
a = []
toxicity = []

for row_index in tqdm(range(len(civil_df_thomas))):
    comment_text = civil_df_thomas.iloc[row_index]['comment_text'].split()
    if civil_df_thomas.iloc[row_index]['toxicity'] < 0.5 and len(comment_text) <= 10 and len(set(train_identities).intersection(comment_text)) != 0:
        identity = str(set(train_identities).intersection(comment_text).pop())
        sentences.append(civil_df_thomas.iloc[row_index]['comment_text'])
        toxicity.append(0)
        cur_a = []
        
        for diff_identity in train_identities:
            cur_a.append(civil_df_thomas.at[row_index, "comment_text"].replace(identity, diff_identity))
        a.append(cur_a)
# print(sentences)
# print(toxicity)
# print(a)


100%|██████████| 448000/448000 [02:05<00:00, 3575.75it/s]

['Your plea will fall upon deaf ears.', 'Transgender soldiers can kill just as efficiently as straight soldiers.', "Women are on strike? I'm gay so who cares.", 'Burn in h3II muslim pos!', "It's tough being a gay guy these days.  LOL", "It figures that you'd know where the gay males are.....;-)", 'Young lady you are delusional. How old are you?', "A female God wouldn't have such asinine rules.", 'Yer transgender phobia is unwarranted fake Beaver.', 'How da heck?Gee`s poor old lady.', "So a life's worth nothing to an old white man", 'a muslim doing muslim stuff', 'How old are you?  Take yer meds.', 'young Muslims will see how barbaric veil wearing is.', '...a society run by male chauvinists.', 'Why do you hate gay people?', 'Gay comedians tell a lot of gay jokes.', 'Utilitas knows this because he is a gay man?', 'Wasilla: Kiddie porn, meth, old timey religion', "he just can't beat the gay guy", 'Come again goof old boy?', 'Haha your such the comedian!!! But a female cat', 'I miss the ol

In [59]:
civil_df_raw = pd.DataFrame(list(zip(*a)))
civil_df = civil_df_raw.T

civil_df.insert(0, column='comment_text', value=sentences)

civil_df

,comment_text,0,1,2,3,4,5,6,7,8,...,25,26,27,28,29,30,31,32,33,34
0,Your plea will fall upon deaf ears.,Your plea will fall upon gay ears.,Your plea will fall upon transgender ears.,Your plea will fall upon trans ears.,Your plea will fall upon queer ears.,Your plea will fall upon lgbt ears.,Your plea will fall upon lgbtq ears.,Your plea will fall upon straight ears.,Your plea will fall upon heterosexual ears.,Your plea will fall upon male ears.,...,Your plea will fall upon old ears.,Your plea will fall upon older ears.,Your plea will fall upon young ears.,Your plea will fall upon younger ears.,Your plea will fall upon teenage ears.,Your plea will fall upon millenial ears.,Your plea will fall upon elderly ears.,Your plea will fall upon blind ears.,Your plea will fall upon deaf ears.,Your plea will fall upon lesbian ears.
1,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...,Transgender soldiers can kill just as efficien...
2,Women are on strike? I'm gay so who cares.,Women are on strike? I'm gay so who cares.,Women are on strike? I'm transgender so who ca...,Women are on strike? I'm trans so who cares.,Women are on strike? I'm queer so who cares.,Women are on strike? I'm lgbt so who cares.,Women are on strike? I'm lgbtq so who cares.,Women are on strike? I'm straight so who cares.,Women are on strike? I'm heterosexual so who c...,Women are on strike? I'm male so who cares.,...,Women are on strike? I'm old so who cares.,Women are on strike? I'm older so who cares.,Women are on strike? I'm young so who cares.,Women are on strike? I'm younger so who cares.,Women are on strike? I'm teenage so who cares.,Women are on strike? I'm millenial so who cares.,Women are on strike? I'm elderly so who cares.,Women are on strike? I'm blind so who cares.,Women are on strike? I'm deaf so who cares.,Women are on strike? I'm lesbian so who cares.
3,Burn in h3II muslim pos!,Burn in h3II gay pos!,Burn in h3II transgender pos!,Burn in h3II trans pos!,Burn in h3II queer pos!,Burn in h3II lgbt pos!,Burn in h3II lgbtq pos!,Burn in h3II straight pos!,Burn in h3II heterosexual pos!,Burn in h3II male pos!,...,Burn in h3II old pos!,Burn in h3II older pos!,Burn in h3II young pos!,Burn in h3II younger pos!,Burn in h3II teenage pos!,Burn in h3II millenial pos!,Burn in h3II elderly pos!,Burn in h3II blind pos!,Burn in h3II deaf pos!,Burn in h3II lesbian pos!
4,It's tough being a gay guy these days. LOL,It's tough being a gay guy these days. LOL,It's tough being a transgender guy these days....,It's tough being a trans guy these days. LOL,It's tough being a queer guy these days. LOL,It's tough being a lgbt guy these days. LOL,It's tough being a lgbtq guy these days. LOL,It's tough being a straight guy these days. LOL,It's tough being a heterosexual guy these days...,It's tough being a male guy these days. LOL,...,It's tough being a old guy these days. LOL,It's tough being a older guy these days. LOL,It's tough being a young guy these days. LOL,It's tough being a younger guy these days. LOL,It's tough being a teenage guy these days. LOL,It's tough being a millenial guy these day

In [60]:
civil_df.to_csv(Path("./data/civil_train_data.csv"))